In [3]:
# import packages
import numpy as np
import pandas as pd
import os
import datetime as dt
import stat
from scipy.stats import pointbiserialr
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
import re


#for text processing
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
nltk.download('wordnet')
import string
nltk.download('words')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
id = '1tkry63EvDXY_CPSfDomBoa8IbUXZN-2y'

In [7]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train-balanced-sarcasm.csv')  
df = pd.read_csv('train-balanced-sarcasm.csv')
# Dataset is now stored in a Pandas Dataframe

In [8]:
df.shape

(1010826, 10)

# 1.Missing comments

In [9]:
len(df[pd.isna(df["comment"])==True].head())

5

In [10]:
#drop rows that has missing comments
df = df.dropna(subset=['comment'])
df.isnull().sum()

label             0
comment           0
author            0
subreddit         0
score             0
ups               0
downs             0
date              0
created_utc       0
parent_comment    0
dtype: int64

# 2. Drop duplicated comments

In [11]:
sum(df.duplicated(keep=False)) 

53

There are 53 duplicated rows!

In [12]:
#df[df.duplicated(keep=False)]

In [13]:
#df[df.duplicated(keep=False)].head()

In [14]:
df[df.duplicated(subset=['comment','author','subreddit','parent_comment'])]

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
10185,1,Shocking,Clyde_Died,todayilearned,2,2,0,2016-09,2016-09-29 17:53:07,TIL materials needed to produce a pair of $199...
12357,1,Said the under 2k post karma guy *expert*,iam4real,youdontsurf,0,-1,-1,2016-11,2016-11-02 10:09:10,This subreddit really does suck.
15649,1,I am very surprised that there is more than 1 ...,PianoRainMelody,ClashRoyale,4,-1,-1,2016-12,2016-12-30 22:47:58,CONGRATS! YOU ARE AN INSPIRATION!
23777,1,Said the under 2k post karma guy *expert*,iam4real,youdontsurf,1,-1,-1,2016-11,2016-11-02 10:09:10,This subreddit really does suck.
40133,0,F,Leoleomeomeo,The_Donald,6,-1,-1,2016-12,2016-12-08 01:23:58,F
...,...,...,...,...,...,...,...,...,...,...
1008663,0,OK,db2,atheism,1,1,0,2011-03,2011-03-08 02:21:40,STOP YELLING
1009033,1,"He is going to go to each state, be elected ki...",cheney_healthcare,politics,3,3,0,2011-01,2011-01-27 13:49:57,What?
1009401,1,"Yea, it's rare that you turn on the news and s...",haiduz,IAmA,5,5,0,2011-07,2011-07-27 21:23:41,"Really, CNN could use any publicity it could g..."
1010325,1,"Probably not without our hero, Sara Palin, bei...",sirjoebob,pics,5,5,0,2009-08,2009-08-17 15:58:06,When can we start hunting them from helicopters?


There are 256 rows with duplicated comments


In [15]:
df = df.drop_duplicates(subset=['comment','author','subreddit','parent_comment'],keep='last')
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


We are left with 1010517 rows.

In [16]:
#df['comment'].dropna

In [17]:
df['comment'] = df['comment'].astype(str)

In [18]:
df['parent_comment'] = df['parent_comment'].astype(str)
df['og_comment'] = df['comment']
df['og_parentcomment'] = df['parent_comment']
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,og_comment,og_parentcomment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",NC and NH.,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,"They were underdogs earlier today, but since G...",They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,I could use one of those tools.,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",I'm sure that Iran and N. Korea have the techn...,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,"whatever you do, don't vote green!",In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,Perhaps this is an atheist conspiracy to make ...,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,The Slavs got their own country - it is called...,I've always been unsettled by that. I hear a l...


# Text preprocessing (for both comment and parent comment)

In [19]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


In [20]:
# 3.Lower case
df['comment'] = df['comment'].str.lower()
df['parent_comment'] = df['parent_comment'].str.lower()

In [21]:
contractions = { 
"ain't":  "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
def expandContractions(sentence):
  if type(sentence) == float:
    quit()
  text = sentence.split()
  new_text = []
  for word in text:
    if word in contractions:
      new_text.append(contractions[word])
    else:
      new_text.append(word)
    text = " ".join(new_text)
  return text
expandContractions("they should've asked" )


'they should have asked'

In [22]:
df['comment'] = df['comment'].apply(expandContractions)
df['parent_comment'] = df['parent_comment'].apply(expandContractions)

In [23]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


In [24]:
# 4.removing stop words
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['parent_comment'] = df['parent_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [25]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


In [26]:
# 5.remove punctuations (dont remove punctuations if extracting emojis)
df['comment'] = df['comment'].str.replace('[{}]'.format(string.punctuation), '')
df['parent_comment'] = df['parent_comment'].str.replace('[{}]'.format(string.punctuation), '')

In [27]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


In [28]:
# 6.removing single character  (only applicable for tf-idf, not sure about doc2vec)
def split(word):
    return [char for char in word]

comment = 'nc nh'
print(split(comment))

def checkingSingle(comment):
    if type(comment) == float:
      quit()
    if len(split(comment)) == 1:
        return 1
    else:
        return comment

df['comment'] = df['comment'].apply(checkingSingle)

['n', 'c', ' ', 'n', 'h']


In [29]:
#df[df['comment'] == 1]

In [30]:
#removing 1313 single charcter comments
df = df[df['comment'] != 1]


In [31]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


In [32]:
#7. Lemmatization 
# change, chaging, changed, changes => change 
lemmatizer = WordNetLemmatizer()

In [33]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,og_comment,og_parentcomment
0,0,nc nh,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,yeah get argument point id prefer lived nc well,NC and NH.,"Yeah, I get that argument. At this point, I'd ..."
1,0,know west teams play west teams east teams right,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,blazers mavericks the wests 5 6 seed even carr...,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...
2,0,underdogs earlier today since gronks announcem...,Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,favored win,"They were underdogs earlier today, but since G...",They're favored to win.
3,0,meme funny none new york nigga ones are,icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass kill buzz,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz
4,0,could use one tools,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,yep confirm saw tool use that made boy easport...,I could use one of those tools.,Yep can confirm I saw the tool they use for th...


In [34]:
df['comment'] = df["comment"].apply(lambda x: ' '.join([lemmatizer.lemmatize(word)  for word in x.split()]))
df['parent_comment'] = df["parent_comment"].apply(lambda x: ' '.join([lemmatizer.lemmatize(word)  for word in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


# Feature extraction

In [37]:
!pip install emot

     |████████████████████████████████| 61 kB 18 kB/s 


In [38]:
#7. Extracting emojis

#counting the number of emojis
# Sarcastic Emoji are ones sometimes used with sarcastic or ironical statements (e.g., ":P").
#!pip install emot
from emot.emo_unicode import EMOTICONS_EMO
all_emoticons = list(EMOTICONS_EMO.keys())

In [39]:
def emoji_counter(sentence):
    count = 0
    for word in sentence.split():
        if word in all_emoticons:
            count += 1
    return count
    

In [40]:
df['emoticon_count'] = df['og_comment'].apply(lambda x: emoji_counter(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
#check proportion of comments with emojis that are labelled 1
proportion = df.loc[(df['emoticon_count'] > 0) & (df['label'] == 1)].size / df.loc[df['emoticon_count'] > 0].size
print(f'proportion of sarcastic comments among all comments that contain emojis: {proportion}')
df.loc[df['emoticon_count'] > 0].size

proportion of sarcastic comments among all comments that contain emojis: 0.20370782350760103


175305

In [42]:
#check most popular emojis used for sarcastic comments
emoji_dict = {emoji:0 for emoji in all_emoticons}
for comment in df.loc[df['label'] == 1]['og_comment']:
    for word in comment.split():
        if word in emoji_dict:
            emoji_dict[word] += 1
            
sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)

[(';)', 495),
 (':(', 487),
 (':)', 464),
 (':D', 318),
 (':P', 224),
 ('XD', 201),
 (':/', 119),
 (':O', 71),
 (":'(", 62),
 ('XP', 54),
 ('D:', 49),
 (':^)', 44),
 (':-)', 40),
 (':3', 31),
 (':o', 23),
 (':c', 18),
 (":')", 17),
 (';_;', 16),
 (':-(', 15),
 ('=)', 13),
 ('QQ', 13),
 (':|', 12),
 (':)))', 9),
 ('=D', 9),
 (';D', 9),
 (';-;', 9),
 (':S', 7),
 (':))', 6),
 (':]', 6),
 ('=]', 5),
 (':@', 4),
 ('DX', 4),
 ('=/', 3),
 ('^_^', 3),
 ('Q.Q', 3),
 (':[', 2),
 (':b', 2),
 (':L', 2),
 (':x', 2),
 ('o.O', 2),
 (':}', 1),
 (':{', 1),
 ('D;', 1),
 (':*', 1),
 (';]', 1),
 (';^)', 1),
 (':$', 1),
 (';n;', 1),
 ('T.T', 1),
 ('oO', 1),
 (':‑)', 0),
 (':-))', 0),
 (':-)))', 0),
 (':-]', 0),
 (':-3', 0),
 (':->', 0),
 (':>', 0),
 ('8-)', 0),
 (':o)', 0),
 (':-}', 0),
 (':c)', 0),
 (':‑D', 0),
 ('8‑D', 0),
 ('8D', 0),
 ('X‑D', 0),
 ('=3', 0),
 ('B^D', 0),
 (':‑(', 0),
 (':‑c', 0),
 (':‑<', 0),
 (':<', 0),
 (':‑[', 0),
 (':-||', 0),
 ('>:[', 0),
 ('>:(', 0),
 (":'‑(", 0),
 (":'‑)", 0),
 (

In [43]:
#8. Extracting exlamation mark
def exclaim_counter(sentence):
    count = 0
    for i in range(len(sentence)):
        if sentence[i] == '!':
            count += 1
    return count

df['exclaim_count'] = df['og_comment'].apply(lambda x: exclaim_counter(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [44]:
#9. Extracting question mark
def qns_mark_counter(sentence):
    count = 0
    for i in range(len(sentence)):
        if sentence[i] == '?':
            count += 1
    return count
df['qns_mark_count'] = df['og_comment'].apply(lambda x: qns_mark_counter(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
#10. Extracting ellipses
def ellipses_counter(sentence):
    count = 0
    for word in sentence.split():
        if '...' in word:
            count += 1
    return count
df['ellipses_mark_count'] = df['og_comment'].apply(lambda x: ellipses_counter(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
#11. Extracting interjection
from nltk import pos_tag

def interjection_counter(sentence):
    count = 0
    tagged_sentence = pos_tag(word_tokenize(sentence))
    for tuples in tagged_sentence:
        if tuples[1] == 'UH':
            count += 1
    return count
df['interjection_count'] = df['og_comment'].apply(lambda x: interjection_counter(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [47]:
#12. Extracting laughter words
import string
import re
def laughter_num(sentence): 
    count = 0
    sentence = re.sub(r'[^\w\s]','',sentence).lower()
    laughter_words = ['lol', 'lhh', 'jk', 'wow', 'kidding', 'ha ha', 'haha', 'rofl', 'roflmao', 'lmao', 'wtf']
    for word in sentence.split(): 
        for lword in laughter_words:
            if lword in word:
                count += 1
                break
    return count
df['laughter_words_count'] = df['og_comment'].apply(lambda x: laughter_num(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
#13. Extracting fully captilatized word
#fully capitalized words
def capitalized_word_counter(sentence):
    capitalized_word_wount = 0
    sentence = re.sub(r'[^\w\s]','',sentence)
    for word in sentence.split():
        if word.isupper() and len(word) > 1:
            capitalized_word_wount += 1
    return capitalized_word_wount

df['capitalized_word_count'] = df['og_comment'].apply(lambda x: capitalized_word_counter(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [49]:
#14. Extracting partially caps words
def partial_cap(sentence):
    count = 0
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    for word in sentence.split():
        if word.isupper():
            continue
        word_count = 0
        for i in range(len(word)):
            if i == 0:
                continue
            if word[i].isupper():
                word_count += 1
        if (word_count > 1 or word_count >= ((len(word))//2)) and word_count > 0:
            count += 1
    return count

In [50]:
df['partial_capitalized_word_count'] = df['og_comment'].apply(lambda x: partial_cap(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Feature engineering (Sentiment Analysis)

In [51]:
!python -m spacy download en_core_web_sm
!pip install vaderSentiment

     |████████████████████████████████| 12.0 MB 15.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 125 kB 15.2 MB/s 


In [52]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [53]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [54]:
def vader(sentence):
  score = analyser.polarity_scores(sentence)
  comp = score['compound']
  if comp>=0.05:
    return 'pos'   # positive
  elif abs(comp)<0.05:
    return 'neu'  # neutral
  elif comp<=-0.05:
    return 'neg'   #negative

In [55]:
df["sentiment"] = df["og_comment"].apply(vader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,og_comment,og_parentcomment,emoticon_count,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capitalized_word_count,sentiment
0,0,nc nh,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,yeah get argument point id prefer lived nc well,NC and NH.,"Yeah, I get that argument. At this point, I'd ...",0,0,0,0,0,0,2,0,neu
1,0,know west team play west team east team right,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,blazer maverick the west 5 6 seed even carry g...,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...,0,0,1,0,0,0,0,0,pos
2,0,underdog earlier today since gronks announceme...,Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,favored win,"They were underdogs earlier today, but since G...",They're favored to win.,0,0,0,0,0,0,0,0,neu
3,0,meme funny none new york nigga one are,icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass kill buzz,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz,0,0,0,0,0,0,0,0,neg
4,0,could use one tool,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,yep confirm saw tool use that made boy easport...,I could use one of those tools.,Yep can confirm I saw the tool they use for th...,0,0,0,0,0,0,0,0,neu


In [57]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,og_comment,og_parentcomment,emoticon_count,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capitalized_word_count,sentiment
0,0,nc nh,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,yeah get argument point id prefer lived nc well,NC and NH.,"Yeah, I get that argument. At this point, I'd ...",0,0,0,0,0,0,2,0,neu
1,0,know west team play west team east team right,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,blazer maverick the west 5 6 seed even carry g...,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...,0,0,1,0,0,0,0,0,pos
2,0,underdog earlier today since gronks announceme...,Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,favored win,"They were underdogs earlier today, but since G...",They're favored to win.,0,0,0,0,0,0,0,0,neu
3,0,meme funny none new york nigga one are,icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass kill buzz,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz,0,0,0,0,0,0,0,0,neg
4,0,could use one tool,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,yep confirm saw tool use that made boy easport...,I could use one of those tools.,Yep can confirm I saw the tool they use for th...,0,0,0,0,0,0,0,0,neu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,im sure iran n korea technology create pigbird...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,one calling engineered pathogen report virus b...,I'm sure that Iran and N. Korea have the techn...,"No one is calling this an engineered pathogen,...",0,0,0,0,0,0,0,0,pos
1010822,1,whatever do vote green,BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,move typical recent donothing approach co2 emi...,"whatever you do, don't vote green!",In a move typical of their recent do-nothing a...,0,1,0,0,0,0,0,0,neu
1010823,1,perhaps atheist conspiracy make christian look...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,screw disabledive got get church time,Perhaps this is an atheist conspiracy to make ...,Screw the Disabled--I've got to get to Church ...,0,0,1,0,0,0,0,0,neg
1010824,1,slav got country called kosovo,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,ive always unsettled that hear lot jewish peop...,The Slavs got their own country - it is called...,I've always been unsettled by that. I hear a l...,0,0,0,0,0,0,0,0,neu


In [58]:
df.to_pickle("complete_df3")

In [59]:
df.to_csv("cleaned2.csv")

In [60]:
print(len(df[pd.isna(df["comment"])]),len(df[pd.isna(df["parent_comment"])]))


0 0


In [61]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,og_comment,og_parentcomment,emoticon_count,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capitalized_word_count,sentiment
0,0,nc nh,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,yeah get argument point id prefer lived nc well,NC and NH.,"Yeah, I get that argument. At this point, I'd ...",0,0,0,0,0,0,2,0,neu
1,0,know west team play west team east team right,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,blazer maverick the west 5 6 seed even carry g...,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...,0,0,1,0,0,0,0,0,pos
2,0,underdog earlier today since gronks announceme...,Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,favored win,"They were underdogs earlier today, but since G...",They're favored to win.,0,0,0,0,0,0,0,0,neu
3,0,meme funny none new york nigga one are,icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass kill buzz,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz,0,0,0,0,0,0,0,0,neg
4,0,could use one tool,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,yep confirm saw tool use that made boy easport...,I could use one of those tools.,Yep can confirm I saw the tool they use for th...,0,0,0,0,0,0,0,0,neu
